In [1]:
import tensorflow as tf
from tensorflow import keras
import externalTensor as exT
import winsound
import numpy as np
from sklearn.model_selection import KFold
from boxScore import boxScore
from codetiming import Timer
import lime
from lime import lime_tabular

# Preparation of Data

In [2]:
years="2021-22"
# stats="traditional"
stats='advance'
box_score=boxScore(years,stats)
x_train, x_test, y_train, y_test=box_score.separation()



# Possible Hyperparameters

In [3]:

activation=['relu','sigmoid']
# activation=['relu']
number_neurons=[10,30,50,100,150,200]
# number_neurons=[10]
possible_learning_rate=[0.0001,0.001,0.01]
# possible_learning_rate=[0.001]

# Greedy Approch

In [4]:
kf = KFold(n_splits=10, random_state=1, shuffle=True)
bestModelArray=[]
for nN in number_neurons:
    bestRes={'val_acc':0}
    for el in possible_learning_rate:
        for act in activation:
                # Initialize model
                model = exT.makeModel(nN,act,len(x_train.columns))
                
                # Instantiate an optimizer to train the model.
                optimizer = keras.optimizers.SGD(learning_rate=el)
                # Instantiate a loss function.
                loss_fn = tf.keras.losses.BinaryCrossentropy()
                res={
                    "learning_rate":el,
                    'num_neurons':nN,
                    'acti_fun':act
                }
                
                model.compile(optimizer=optimizer,loss=loss_fn,metrics=['accuracy'])

                res=exT.trainBasic(res,x_train,y_train,
                    x_test, y_test, 
                    model,
                    kf,
                    500
                   
                )
                
                #Saved the best model based on the accuracy
                if(res['val_acc']>bestRes['val_acc']):
                    bestModel=model
                    bestRes=res
                        
                    
                
    print("Best model",bestRes)
    # bestModelArray.append(bestModel)
    # f = open("saved_model_"+stats+"/summary.txt", "a")
    # tmpName=str(bestRes['num_neurons'])+"_"+str(bestRes['acti_fun'])+"_"+str(bestRes['learning_rate'])+"_LOSS_"+str(bestRes['loss'])+"_ACC_"+str(bestRes['acc'])+"_LOSSVAL_"+str(bestRes['val_loss'])+"_ACCVAL_"+str(bestRes['val_acc'])  
    # f.write(tmpName+"\n")
    # f.close()


Best model {'learning_rate': 0.01, 'num_neurons': 10, 'acti_fun': 'relu', 'loss': 0.0707, 'acc': 0.9917, 'val_loss': 1.889, 'val_acc': 0.614}
Best model {'learning_rate': 0.0001, 'num_neurons': 30, 'acti_fun': 'relu', 'loss': 0.065, 'acc': 0.9917, 'val_loss': 0.9441, 'val_acc': 0.62}
Best model {'learning_rate': 0.0001, 'num_neurons': 50, 'acti_fun': 'relu', 'loss': 0.1107, 'acc': 0.975, 'val_loss': 0.7596, 'val_acc': 0.656}
Best model {'learning_rate': 0.001, 'num_neurons': 100, 'acti_fun': 'relu', 'loss': 0.0261, 'acc': 0.995, 'val_loss': 1.2794, 'val_acc': 0.616}
Best model {'learning_rate': 0.001, 'num_neurons': 150, 'acti_fun': 'sigmoid', 'loss': 0.1497, 'acc': 1.0, 'val_loss': 0.7254, 'val_acc': 0.64}
Best model {'learning_rate': 0.001, 'num_neurons': 200, 'acti_fun': 'sigmoid', 'loss': 0.1667, 'acc': 0.9917, 'val_loss': 0.7401, 'val_acc': 0.618}


Best model {'learning_rate': 0.001, 'num_neurons': 10, 'acti_fun': 'sigmoid', 'loss': 0.5643, 'acc': 0.7683, 'val_loss': 0.6574, 'val_acc': 0.63}
Best model {'learning_rate': 0.01, 'num_neurons': 30, 'acti_fun': 'sigmoid', 'loss': 0.3488, 'acc': 0.8533, 'val_loss': 0.734, 'val_acc': 0.628}
Best model {'learning_rate': 0.001, 'num_neurons': 50, 'acti_fun': 'relu', 'loss': 0.3494, 'acc': 0.8483, 'val_loss': 0.7065, 'val_acc': 0.628}
Best model {'learning_rate': 0.001, 'num_neurons': 100, 'acti_fun': 'relu', 'loss': 0.3444, 'acc': 0.8483, 'val_loss': 0.702, 'val_acc': 0.638}
Best model {'learning_rate': 0.001, 'num_neurons': 150, 'acti_fun': 'sigmoid', 'loss': 0.458, 'acc': 0.815, 'val_loss': 0.6561, 'val_acc': 0.666}
Best model {'learning_rate': 0.01, 'num_neurons': 200, 'acti_fun': 'sigmoid', 'loss': 0.3282, 'acc': 0.855, 'val_loss': 0.7237, 'val_acc': 0.624

# Save Model

In [18]:
# bestModelArray[2].save("saved_model_"+stats+"/"+"bestModel")
# model = tf.keras.models.load_model('saved_model_'+stats+'/bestModel')


INFO:tensorflow:Assets written to: saved_model_advance/bestModel\assets


# LIME 

In [ ]:

explainer = lime_tabular.LimeTabularExplainer(
                        training_data=np.array(x_train),
                        feature_names=x_train.columns,
                        class_names=['win first team', 'win second team'],
                        mode='classification'
                    )
answer=[]
r=np.asarray(x_test)
for i in range(len(x_test)):

    tmp=model.predict(r[i][None,...])
    # [1,0] (win first team ) and [0,1] (win second team)
    boolean_vector=np.rint(tmp)==y_test[i]

    if(boolean_vector.all()):
            
        exp = explainer.explain_instance(
            data_row=x_test.iloc[i], 
            predict_fn=model.predict
        )
      
        answer.append(exp.as_list())
# exp.show_in_notebook(show_table=True)



In [35]:



conclusion=[]
absvalue=[]
counts={}
oblio=0
for list in answer:
    for el in list[0:4]:
        
        stat=[word for word in el[0].split() if word[0].isalpha()] 
        conclusion.append([stat[0],abs(el[1])])
        if stat[0]  not in counts: 
            counts[stat[0]]=abs(el[1])
        else:
            
            counts[stat[0]]=counts.get(stat[0])+abs(el[1])

    oblio=1


sortdict = dict(sorted(counts.items(), key=lambda x:x[1],reverse=True))


In [ ]:
sortdict